# GPT for Linguistic Annotation

In [1]:
# imports for this project
import gspread
import pandas as pd
import os
from utils import append_data, lm_completion

### Sys-admin

In [2]:
# Google Sheets API
gc = gspread.service_account(filename='fignews-7b178eec49aa.json')
SHEET_ID = "1W3tCrA14vD0WyKC79tPplUvircS9OttzYOkz9BQwljk"
SHEET_NAME = "Main"

Get Sample Data from Google Sheets

In [3]:
spreadsheet = gc.open_by_key(SHEET_ID)
worksheet = spreadsheet.worksheet(SHEET_NAME)
rows = worksheet.get_all_records()
df = pd.DataFrame(rows)
print(df.head())
sample = df.sample(5)

  Batch Source Language  ID  Type  \
0   B01         English   1  MAIN   
1   B01         English   2  MAIN   
2   B01         English   4  MAIN   
3   B01         English   7  MAIN   
4   B01         English   8  MAIN   

                                                Text  \
0  Yemen's Houthis have waded into the Israel-Ham...   
1             Isreal - Hamas Conflict | Face to Face   
2  Videos show how armed men from Gaza stormed a ...   
3  Protest in Aligarh Muslim University in suppor...   
4  IDF releases audio recording about misfired ro...   

                                          English MT  \
0  Yemen's Houthis have waded into the Israel-Ham...   
1             Isreal - Hamas Conflict | Face to Face   
2  Videos show how armed men from Gaza stormed a ...   
3  Protest in Aligarh Muslim University in suppor...   
4  IDF releases audio recording about misfired ro...   

                                           Arabic MT Annotator ID      Bias  \
0  خاض الحوثيون في اليمن

In [4]:
def apply_tag(df):
    
    prompt: str = """
    You are an expert for linguistic media bias detection. You are asked to label the following text with one of the following labels:
    Unbiased
    Biased against Palestine
    Biased against Israel
    Biased against both Palestine and Israel
    Biased against others
    Unclear
    Not Applicable.
    There is not more context to include that what is given to you, so please make your best judgment based on the text alone. Just reply with the label. Here is the text:
    """
    df['GPT proposal'] = df['English MT'].apply(lambda row: lm_completion([
        {"role": "system", "content": prompt},
        {"role": "system", "content": row}]))
    return df

df_annotated = apply_tag(sample)
print(df_annotated.head())

      Batch Source Language     ID  Type  \
2149    B03          Arabic  14521  MAIN   
1833    B03         English    673  MAIN   
309     B01          Arabic   3530  MAIN   
11796   B14         English   3727  MAIN   
12011   B14          Arabic   9325  MAIN   

                                                    Text  \
2149   "فتح": تصريحات المدعو حماد تعبر عن انحطاط وطني...   
1833   HAPPENING NOW: Pres. Biden delivers remarks af...   
309    بدأ عملها بخطف جلعاد شاليط، وكانت المسؤولة عن ...   
11796  The Republican presidential candidates all cal...   
12011  منذ 15 عاماً.. 4 حروب اندلعت بين الفلسطينيين و...   

                                              English MT  \
2149   "Fatah": The statements of the so-called Hamma...   
1833   HAPPENING NOW: Pres. Biden delivers remarks af...   
309    Her work began with the kidnapping of Gilad Sh...   
11796  The Republican presidential candidates all cal...   
12011  15 years ago... 4 wars broke out between Pales...   

             

In [5]:
df_annotated.to_excel("annotated.xlsx")

In [6]:
def tag_connotations(df):
    prompt: str = """
    You are an expert for linguistic media bias detection. You are asked to label words in the following text with one of the following labels:
    [negative]: meaning that the word has a negative connotation
    [positive]: meaning that the word has a positive connotation
    [factuality]: meaning that the word adds doubt to the factuality of the text
    Directly apply the tag behin the word it applies to, for example:
    This is an awful[negative] text.
    Do not change anything else about the text, just add the tags, where applicable. Here is the text:
    """
    df['GPT connotation'] = df['English MT'].apply(lambda row: lm_completion([
        {"role": "system", "content": prompt},
        {"role": "system", "content": row}]))
    return df

df_annotated_connotations = tag_connotations(df_annotated)
print(df_annotated_connotations.head())

      Batch Source Language     ID  Type  \
2149    B03          Arabic  14521  MAIN   
1833    B03         English    673  MAIN   
309     B01          Arabic   3530  MAIN   
11796   B14         English   3727  MAIN   
12011   B14          Arabic   9325  MAIN   

                                                    Text  \
2149   "فتح": تصريحات المدعو حماد تعبر عن انحطاط وطني...   
1833   HAPPENING NOW: Pres. Biden delivers remarks af...   
309    بدأ عملها بخطف جلعاد شاليط، وكانت المسؤولة عن ...   
11796  The Republican presidential candidates all cal...   
12011  منذ 15 عاماً.. 4 حروب اندلعت بين الفلسطينيين و...   

                                              English MT  \
2149   "Fatah": The statements of the so-called Hamma...   
1833   HAPPENING NOW: Pres. Biden delivers remarks af...   
309    Her work began with the kidnapping of Gilad Sh...   
11796  The Republican presidential candidates all cal...   
12011  15 years ago... 4 wars broke out between Pales...   

             

In [7]:
df_annotated_connotations.to_excel("annotated_connotations.xlsx")

Update Google Sheets with our Results

In [8]:
append_data(df_annotated_connotations[["GPT proposal", "GPT connotation"]], worksheet, 2)